In [ ]:
import matplotlib.pyplot as plt

from starter_code import (
    ani1_config,
    load_ani1_data,
    calc_resid,
    compute_rmse_by_num_heavy_atoms,
    plot_rmse_by_num_heavy_atoms,
    compute_rmse_by_num_bonds,
    plot_rmse_by_num_bonds,
    compute_rmse_by_num_atoms,
    plot_rmse_by_num_atoms,
    isin_tuple_series
)
import pandas as pd

In [ ]:
molecules = load_ani1_data()

# Calculate the residual vector for each method-method combination
resid = calc_resid(molecules)

rmse_df = compute_rmse_by_num_heavy_atoms(molecules, resid, ani1_config["heavy_atoms"])

In [ ]:
fig, ax = plt.subplots(figsize=(11, 8.5))

ax.set_prop_cycle(
    color=["b", "r", "g", "b", "r", "g"],
    linestyle=["-", "-", "-", "-.", "-.", "-."],
    marker=["o", "o", "o", "o", "o", "o"],
)

rmse_df_plot = rmse_df[rmse_df["Method Pair"] == ("dt", "dt")][
    [
        "RMSE",
        "RMSE / sqrt(nh)",
        "RMSE(E/nh)",
        "MAE",
        "MAE / sqrt(nh)",
        "MAE(E/nh)",
    ]
]

plt.plot(list(range(1, 9)), rmse_df_plot.to_numpy())

plt.legend(
    [
        "RMSE(E)",
        "RMSE(E) / sqrt(nh)",
        "RMSE(E/nh)",
        "MAE(E)",
        "MAE(E) / sqrt(nh)",
        "MAE(E/nh)",
    ],
    loc="upper left",
    fancybox=True,
)

In [ ]:
rmse_df_plot

In [ ]:
# Will produce 91 plots, one for each method-method combination
# plot_rmse_by_num_heavy_atoms(rmse_df)

# Will produce a plot for each dftb-method combination
# plot_rmse_by_num_heavy_atoms(
#     rmse_df[isin_tuple_series("dt", rmse_df["Method Pair"])],
#     method_id_to_name=ani1_config["target"],
# )

In [ ]:
rmse_df_nbonds = compute_rmse_by_num_bonds(molecules, resid)

In [ ]:
# plot_rmse_by_num_bonds(
#     rmse_df_nbonds[isin_tuple_series("dt", rmse_df_nbonds["Method Pair"])],
#     method_id_to_name=ani1_config["target"],
# )

# RMSE vs. number of heavy atoms plots

In [ ]:
for (method1, method2), group in rmse_df.groupby("Method Pair"):
    method1_full_name = ani1_config["target"][method1]
    method2_full_name = ani1_config["target"][method2]

    # fig, ax = plt.subplots(nrows=1, ncols=2, sharey=True, figsize=(10, 5))
    # fig.suptitle(f"{method1_full_name} vs. {method2_full_name}")

    title = f"RMSE vs. # of Heavy Atoms ({method1_full_name} & {method2_full_name})"
    group.set_index("Heavy Atoms")[["RMSE", "RMSE / sqrt(nh)"]].plot(title=title)

    # group = rmse_df_nbonds_groupby.get_group((method1, method2))
    # group = group[group["Bonds"] != 0]
    # title = f"RMSE vs. # of Bonds (Approximate)"
    # group.set_index("Bonds").sort_index()[["RMSE", "RMSE / sqrt(nbonds)"]].plot(
    #     title=title, ax=ax[1]
    # )

    #plt.show()
    plt.savefig(f"../without_mae/error_vs_atoms/{method1}_{method2}.png", dpi=300)

# RMSE vs. number of bonds plots

In [ ]:
rmse_df_nbonds_groupby = rmse_df_nbonds.groupby("Method Pair")
for (method1, method2), group in rmse_df.groupby("Method Pair"):
    method1_full_name = ani1_config["target"][method1]
    method2_full_name = ani1_config["target"][method2]

    # fig, ax = plt.subplots(nrows=1, ncols=2, sharey=True, figsize=(10, 5))
    # fig.suptitle(f"{method1_full_name} vs. {method2_full_name}")
    # title = f"RMSE vs. # of Heavy Atoms {method1_full_name} & {method2_full_name}"
    # group.set_index("Heavy Atoms")[["RMSE", "RMSE / sqrt(nh)"]].plot(title=title)

    group = rmse_df_nbonds_groupby.get_group((method1, method2))
    group = group[group["Bonds"] != 0]
    title = f"RMSE vs. # of Bonds (Approximate) ({method1_full_name} & {method2_full_name})"
    group.set_index("Bonds").sort_index()[["RMSE", "RMSE / sqrt(nbonds)"]].plot(
        title=title
    )

    # plt.show()
    plt.savefig(f"../plots/error_vs_bonds/{method1}_{method2}.png", dpi=300)

# Side-by-side RMSE vs. atom count and bond count plots

In [ ]:
rmse_df_nbonds_groupby = rmse_df_nbonds.groupby("Method Pair")
for (method1, method2), group in rmse_df.groupby("Method Pair"):
    method1_full_name = ani1_config["target"][method1]
    method2_full_name = ani1_config["target"][method2]

    fig, ax = plt.subplots(nrows=1, ncols=2, sharey=True, figsize=(10, 5))
    fig.suptitle(f"{method1_full_name} vs. {method2_full_name}")

    title = f"RMSE vs. # of Heavy Atoms"
    group.set_index("Heavy Atoms")[["RMSE", "RMSE / sqrt(nh)"]].plot(title=title, ax=ax[0])

    group = rmse_df_nbonds_groupby.get_group((method1, method2))
    group = group[group["Bonds"] != 0]
    title = f"RMSE vs. # of Bonds (Approximate)"
    group.set_index("Bonds").sort_index()[["RMSE", "RMSE / sqrt(nbonds)"]].plot(
        title=title, ax=ax[1]
    )

    # plt.show()
    plt.savefig(f"../plots/side_by_side/{method1}_{method2}.png", dpi=300)

# Side-by-side RMSE vs. Heavy Atom and Atom

In [ ]:
import os


os.makedirs(f"../without_mae/side_by_side-atoms_heavyatoms/", exist_ok=True)

In [ ]:
rmse_natoms = compute_rmse_by_num_atoms(molecules, resid)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def test_rmse(n, mean, var):
    stdev = np.sqrt(var)
    rng = np.random.default_rng(42)
    x_1 = np.linspace(0, 10, n)
    x_2 = x_1 + rng.normal(loc=mean, scale=stdev, size=n)

    error = x_1 - x_2

    rmse_1 = np.mean(error**2)**0.5 / var**0.5
    rmse_2 = np.mean((error/var)**2)**0.5

    mae_1 = np.mean(np.abs(error)) / var**0.5
    mae_2 = np.mean(np.abs(error) / var**0.5)

    print(rmse_1, rmse_2)
    print(mae_1, mae_2)

test_rmse(100000, 0, 3)

### MAE(E/n)

In [ ]:
rmse_df_natoms_groupby = rmse_natoms.groupby("Method Pair")
for (method1, method2), group in rmse_df.groupby("Method Pair"):
    method1_full_name = ani1_config["target"][method1]
    method2_full_name = ani1_config["target"][method2]

    fig, ax = plt.subplots(nrows=1, ncols=2, sharey=True, figsize=(10, 5))
    fig.suptitle(f"{method1_full_name} vs. {method2_full_name}")

    title = f"MAE vs. # of Heavy Atoms"
    group.set_index("Heavy Atoms")[["MAE", "RMSE(E/nh)"]].plot(title=title, ax=ax[0])

    group = rmse_df_natoms_groupby.get_group((method1, method2))
    title = f"MAE vs. # of Atoms"
    group.set_index("Atoms").sort_index()[["MAE", "RMSE(E/n)"]].plot(
        title=title, ax=ax[1]
    )

    # plt.show()
    plt.savefig(f"../without_mae/side_by_side-atoms_heavyatoms-2/{method1}_{method2}.png", dpi=300)

### RMSE(E/n)

In [ ]:
rmse_df_natoms_groupby = rmse_natoms.groupby("Method Pair")
for (method1, method2), group in rmse_df.groupby("Method Pair"):
    method1_full_name = ani1_config["target"][method1]
    method2_full_name = ani1_config["target"][method2]

    fig, ax = plt.subplots(nrows=1, ncols=2, sharey=True, figsize=(10, 5))
    fig.suptitle(f"{method1_full_name} vs. {method2_full_name}")

    title = f"RMSE vs. # of Heavy Atoms"
    group.set_index("Heavy Atoms")[["RMSE", "RMSE(E/nh)"]].plot(title=title, ax=ax[0])

    group = rmse_df_natoms_groupby.get_group((method1, method2))
    title = f"RMSE vs. # of Atoms"
    group.set_index("Atoms").sort_index()[["RMSE", "RMSE(E/n)"]].plot(
        title=title, ax=ax[1]
    )
    
    ax[0].set_ylim(0, 20)

    # plt.savefig(f"../without_mae/side_by_side-atoms_heavyatoms-2/{method1}_{method2}.png", dpi=300)
    plt.show(); break

### RMSE(E) / n

In [ ]:
rmse_df_natoms_groupby = rmse_natoms.groupby("Method Pair")
for (method1, method2), group in rmse_df.groupby("Method Pair"):
    method1_full_name = ani1_config["target"][method1]
    method2_full_name = ani1_config["target"][method2]

    fig, ax = plt.subplots(nrows=1, ncols=2, sharey=True, figsize=(10, 5))
    fig.suptitle(f"{method1_full_name} vs. {method2_full_name}")

    title = f"RMSE vs. # of Heavy Atoms"
    group.set_index("Heavy Atoms")[["RMSE", "RMSE / sqrt(nh)"]].plot(title=title, ax=ax[0])

    group = rmse_df_natoms_groupby.get_group((method1, method2))
    title = f"RMSE vs. # of Atoms"
    group.set_index("Atoms").sort_index()[["RMSE", "RMSE / sqrt(n_atoms)"]].plot(
        title=title, ax=ax[1]
    )
    ax[0].set_ylim(0, 20)

    # plt.savefig(f"../without_mae/side_by_side-atoms_heavyatoms/{method1}_{method2}.png", dpi=300); plt.show()
    plt.show(); break

# Bond Counting Example

In [ ]:
molecule = molecules[100622]
molecule["name"], molecule["iconfig"], molecule["atomic_numbers"]

In [ ]:
from scipy.spatial.distance import cdist
import numpy as np

atom_types = {1: 'H', 6: 'C', 7: 'N', 8: 'O'}

ATOM_PAIR_TO_BOND_ANGSTROM = {
    frozenset([1, 6]): (0.95, 1.70),
    frozenset([1, 7]): (0.95, 1.50),
    frozenset([1, 8]): (0.90, 1.50),
    frozenset([6, 6]): (1.00, 1.90),
    frozenset([6, 7]): (1.10, 1.80),
    frozenset([6, 8]): (1.10, 1.75),
}

def bonds_from_coordinates(coordinates, atomic_numbers):
    pairwise_distance = cdist(coordinates, coordinates)

    bonds = []
    # Loop through combinations of atoms in the molecule
    # Only need to look at one half of the (symmetric) pairwise distance matrix
    # And we don't care about the diagonal either
    for i, j in zip(*np.triu_indices_from(pairwise_distance, k=1)):
        atom_atom_distance = pairwise_distance[i, j]
        atomic_number_pair = frozenset([atomic_numbers[i], atomic_numbers[j]])
        bond_length_min_max = ATOM_PAIR_TO_BOND_ANGSTROM.get(atomic_number_pair)

        # If we have the current atom-atom pair in the bond length lookup table, 
        # check if the distance is within the allowed range
        if bond_length_min_max is not None:
            bond_length_min, bond_length_max = bond_length_min_max
            if bond_length_min < atom_atom_distance < bond_length_max:
                bonds.append((i, j))

    return bonds

In [ ]:
from math import dist

atomic_numbers = molecule["atomic_numbers"]
coordinates = molecule["coordinates"]

bonds = bonds_from_coordinates(coordinates, atomic_numbers)

bond_numbers = [(atomic_numbers[b[0]], atomic_numbers[b[1]]) for b in bonds]
bond_symbols = [(atom_types[n[0]], atom_types[n[1]]) for n in bond_numbers]
bond_distances = [dist(coordinates[b[0]], coordinates[b[1]]) for b in bonds]

pd.DataFrame({"Atom Indices": bonds, "Atomic Numbers": bond_numbers, "Bond Symbols": bond_symbols, "Bond Distance": bond_distances})

In [ ]:
from rdkit import Chem

In [ ]:
# N-(diaminomethylidene)acetamide -- Canonical SMILES
m = Chem.MolFromSmiles('CC(=O)N=C(N)N')
m

In [ ]:
for bond in m.GetBonds():
    print(f"{bond.GetBeginAtom().GetSymbol()}-{bond.GetEndAtom().GetSymbol()} {bond.GetBondType()} bond")

Our bond detection seemed to have missed the C-C bond.

We should expect one of the carbon atom pairs in the distance matrix to have a pairwise distance roughly between 1.2 and 1.54. However, the distances between the carbon atoms are 2.44 and 2.49 from the ANI-1 data (see the first few entries in the table below).

In [ ]:
# Printing the distance matrix
print("Atomic #:", "     ".join([str(x) for x in molecule["atomic_numbers"]]))
for row in cdist(molecule["coordinates"], molecule["coordinates"]):
    print("         ", "  ".join([f"{x:.2f}"for x in row]))